# Chapter 5: Building Your First Hugging Face Dataset

In [ ]:
!pip install transformers datasets

## 5.2 Learning Objectives

By the end of this chapter, you should be able to:
- build and use Hugging Face Datasets
- understand the role of batch normalization in deep learning models
- assess different alternatives for training models using higher-level libraries

## 5.3 A New Dataset

In this chapter, and in the second lab, we'll use a different dataset: [100,000 UK Used Car Dataset](https://www.kaggle.com/datasets/adityadesai13/used-car-dataset-ford-and-mercedes) from Kaggle. It contains scraped data of used car listings split into CSV files according to the manufacturer: Audi, BMW, Ford, Hyundai, Mercedes, Skoda, Toyota, Vauxhall, and VW. It also contains a few extra files of particular models (`cclass.csv`, `focus.csv`, `unclean_cclass.csv`, and `unclean_focus.csv`) that we won't be using.

Each file has nine columns with the car's attributes: model, year, price, transmission, mileage, fuel type, road tax, fuel consumption (mpg), and engine size. Transmission, fuel type, and year are discrete/categorical attributes, the others are continous. Our goal here is to predict the car's price based on its other attributes.

In [1]:
!wget https://github.com/dvgodoy/assets/raw/main/PyTorchInPractice/data/100KUsedCar/car_prices.zip
!unzip car_prices.zip -d car_prices

--2024-09-09 17:10:15--  https://github.com/dvgodoy/assets/raw/main/PyTorchInPractice/data/100KUsedCar/car_prices.zip
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/dvgodoy/assets/main/PyTorchInPractice/data/100KUsedCar/car_prices.zip [following]
--2024-09-09 17:10:16--  https://raw.githubusercontent.com/dvgodoy/assets/main/PyTorchInPractice/data/100KUsedCar/car_prices.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1152744 (1.1M) [application/zip]
Saving to: ‘car_prices.zip’

car_prices.zip      100%[===================>]   1.10M  --.-KB/s    in 0.1s    

2024-09-09 17:10:16 (8.26 MB/s) - 

In [2]:
import os

def filter_for_data(filename):
    return ("unclean" not in filename) and ("focus" not in filename) and ("cclass" not in filename) and filename.endswith(".csv")

folder = './car_prices'
data_files = sorted([os.path.join(folder, fname) 
                     for fname in os.listdir(folder) 
                     if filter_for_data(fname)])
data_files

['./car_prices/audi.csv',
 './car_prices/bmw.csv',
 './car_prices/ford.csv',
 './car_prices/hyundi.csv',
 './car_prices/merc.csv',
 './car_prices/skoda.csv',
 './car_prices/toyota.csv',
 './car_prices/vauxhall.csv',
 './car_prices/vw.csv']

### 5.3.1 Hugging Face Datasets

Our goal is to build a datasets that returns a dictionary with three keys in it: `label` (containing the prices we want to predict), `cont_X` (an array of the continuous attributes), and `cat_X` (an array of sequentially-encoded categorical attributes).

#### 5.3.1.1 Loading CSV Files

![](https://raw.githubusercontent.com/dvgodoy/assets/main/PyTorchInPractice/images/ch0/data_step1.png)

In [3]:
from datasets import load_dataset, Split

colnames = ['model', 'year', 'price', 'transmission', 'mileage', 'fuel_type', 'road_tax', 'mpg', 'engine_size', 'manufacturer']

dataset = load_dataset(path="csv",
                       data_files=data_files, 
                       sep=',', 
                       skiprows=1, 
                       column_names=colnames,
                       split=Split.ALL)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /home/dvgodoy/.cache/huggingface/datasets/csv/default-855731431ffcb4e3/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


In [4]:
dataset.features, dataset.num_columns, dataset.shape

({'model': Value(dtype='string', id=None),
  'year': Value(dtype='int64', id=None),
  'price': Value(dtype='int64', id=None),
  'transmission': Value(dtype='string', id=None),
  'mileage': Value(dtype='int64', id=None),
  'fuel_type': Value(dtype='string', id=None),
  'road_tax': Value(dtype='int64', id=None),
  'mpg': Value(dtype='float64', id=None),
  'engine_size': Value(dtype='float64', id=None),
  'manufacturer': Value(dtype='float64', id=None)},
 10,
 (99187, 10))

In [5]:
dataset[:3]

{'model': [' A1', ' A6', ' A1'],
 'year': [2017, 2016, 2016],
 'price': [12500, 16500, 11000],
 'transmission': ['Manual', 'Automatic', 'Manual'],
 'mileage': [15735, 36203, 29946],
 'fuel_type': ['Petrol', 'Diesel', 'Petrol'],
 'road_tax': [150, 20, 30],
 'mpg': [55.4, 64.2, 55.4],
 'engine_size': [1.4, 2.0, 1.4],
 'manufacturer': [None, None, None]}

In [6]:
dataset['transmission']

['Manual',
 'Automatic',
 'Manual',
 'Automatic',
 'Manual',
 'Automatic',
 'Automatic',
 'Manual',
 'Manual',
 'Manual',
 'Manual',
 'Automatic',
 'Manual',
 'Manual',
 'Manual',
 'Automatic',
 'Automatic',
 'Automatic',
 'Automatic',
 'Manual',
 'Automatic',
 'Automatic',
 'Automatic',
 'Automatic',
 'Automatic',
 'Automatic',
 'Automatic',
 'Manual',
 'Automatic',
 'Manual',
 'Automatic',
 'Manual',
 'Automatic',
 'Automatic',
 'Automatic',
 'Automatic',
 'Manual',
 'Automatic',
 'Manual',
 'Manual',
 'Manual',
 'Automatic',
 'Automatic',
 'Automatic',
 'Automatic',
 'Manual',
 'Automatic',
 'Automatic',
 'Automatic',
 'Automatic',
 'Manual',
 'Automatic',
 'Manual',
 'Manual',
 'Automatic',
 'Automatic',
 'Automatic',
 'Manual',
 'Manual',
 'Manual',
 'Manual',
 'Manual',
 'Manual',
 'Manual',
 'Manual',
 'Manual',
 'Manual',
 'Manual',
 'Automatic',
 'Manual',
 'Manual',
 'Manual',
 'Manual',
 'Manual',
 'Manual',
 'Manual',
 'Automatic',
 'Automatic',
 'Manual',
 'Manual',
 'Manu

In [7]:
train_test = dataset.train_test_split(train_size=0.8)
train_test

DatasetDict({
    train: Dataset({
        features: ['model', 'year', 'price', 'transmission', 'mileage', 'fuel_type', 'road_tax', 'mpg', 'engine_size', 'manufacturer'],
        num_rows: 79349
    })
    test: Dataset({
        features: ['model', 'year', 'price', 'transmission', 'mileage', 'fuel_type', 'road_tax', 'mpg', 'engine_size', 'manufacturer'],
        num_rows: 19838
    })
})

In [8]:
val_test = train_test['test'].train_test_split(train_size=0.5)
val_test

DatasetDict({
    train: Dataset({
        features: ['model', 'year', 'price', 'transmission', 'mileage', 'fuel_type', 'road_tax', 'mpg', 'engine_size', 'manufacturer'],
        num_rows: 9919
    })
    test: Dataset({
        features: ['model', 'year', 'price', 'transmission', 'mileage', 'fuel_type', 'road_tax', 'mpg', 'engine_size', 'manufacturer'],
        num_rows: 9919
    })
})

In [9]:
from datasets import DatasetDict
datasets = DatasetDict({'train': train_test['train'],  # training set from first split
                       'val': val_test['train'],      # test set from first split, split further and renamed
                       'test': val_test['test']})     # test set from first split, split further
datasets

DatasetDict({
    train: Dataset({
        features: ['model', 'year', 'price', 'transmission', 'mileage', 'fuel_type', 'road_tax', 'mpg', 'engine_size', 'manufacturer'],
        num_rows: 79349
    })
    val: Dataset({
        features: ['model', 'year', 'price', 'transmission', 'mileage', 'fuel_type', 'road_tax', 'mpg', 'engine_size', 'manufacturer'],
        num_rows: 9919
    })
    test: Dataset({
        features: ['model', 'year', 'price', 'transmission', 'mileage', 'fuel_type', 'road_tax', 'mpg', 'engine_size', 'manufacturer'],
        num_rows: 9919
    })
})

#### 5.3.1.2 Encoding Categorical Attributes

![](https://raw.githubusercontent.com/dvgodoy/assets/main/PyTorchInPractice/images/ch0/data_step3.png)

In [10]:
datasets['train'].unique('fuel_type')

Flattening the indices:   0%|          | 0/79349 [00:00<?, ? examples/s]

['Diesel', 'Petrol', 'Hybrid', 'Other', 'Electric']

In [11]:
cont_attr = ['year', 'mileage', 'road_tax', 'mpg', 'engine_size']
cat_attr = ['model', 'transmission', 'fuel_type']

def gen_encoder_dict(dataset, col):
    values = sorted(dataset.unique(col))
    values += ['UNKNOWN']
    return dict(zip(values, range(len(values))))

dropdown_encoders = {col: gen_encoder_dict(datasets['train'], col) for col in cat_attr}

Loading cached processed dataset at /home/dvgodoy/.cache/huggingface/datasets/csv/default-855731431ffcb4e3/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-940e91da977fb3f9.arrow
Loading cached processed dataset at /home/dvgodoy/.cache/huggingface/datasets/csv/default-855731431ffcb4e3/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-940e91da977fb3f9.arrow
Loading cached processed dataset at /home/dvgodoy/.cache/huggingface/datasets/csv/default-855731431ffcb4e3/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-940e91da977fb3f9.arrow


In [12]:
dropdown_encoders['fuel_type']

{'Diesel': 0,
 'Electric': 1,
 'Hybrid': 2,
 'Other': 3,
 'Petrol': 4,
 'UNKNOWN': 5}

#### 5.3.1.3 Row Output

In [13]:
datasets['train'][0]

{'model': ' 3 Series',
 'year': 2019,
 'price': 21998,
 'transmission': 'Semi-Auto',
 'mileage': 7237,
 'fuel_type': 'Diesel',
 'road_tax': 145,
 'mpg': 60.1,
 'engine_size': 2.0,
 'manufacturer': None}

In [14]:
import numpy as np

def preproc(row):
    colnames = ['model', 'year', 'price', 'transmission', 'mileage', 'fuel_type', 'road_tax', 'mpg', 'engine_size']#, 'manufacturer']
    
    cat_attr = ['model', 'transmission', 'fuel_type']#, 'manufacturer']
    cont_attr = ['year', 'mileage', 'road_tax', 'mpg', 'engine_size']
    target = 'price'
    
    cont_X = [float(row[name]) for name in cont_attr]
    cat_X = [dropdown_encoders[name].get(row[name], dropdown_encoders[name]['UNKNOWN']) for name in cat_attr]
            
    return {'label': np.array([float(row[target])], dtype=np.float32),
            'cont_X': np.array(cont_X, dtype=np.float32), 
            'cat_X': np.array(cat_X, dtype=int)}

![](https://raw.githubusercontent.com/dvgodoy/assets/main/PyTorchInPractice/images/ch0/data_step4.png)

In [16]:
datasets = datasets.map(preproc)
datasets

Map:   0%|          | 0/79349 [00:00<?, ? examples/s]

Map:   0%|          | 0/9919 [00:00<?, ? examples/s]

Map:   0%|          | 0/9919 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['model', 'year', 'price', 'transmission', 'mileage', 'fuel_type', 'road_tax', 'mpg', 'engine_size', 'manufacturer', 'label', 'cont_X', 'cat_X'],
        num_rows: 79349
    })
    val: Dataset({
        features: ['model', 'year', 'price', 'transmission', 'mileage', 'fuel_type', 'road_tax', 'mpg', 'engine_size', 'manufacturer', 'label', 'cont_X', 'cat_X'],
        num_rows: 9919
    })
    test: Dataset({
        features: ['model', 'year', 'price', 'transmission', 'mileage', 'fuel_type', 'road_tax', 'mpg', 'engine_size', 'manufacturer', 'label', 'cont_X', 'cat_X'],
        num_rows: 9919
    })
})

In [17]:
datasets = datasets.select_columns(['label', 'cont_X', 'cat_X'])
datasets

DatasetDict({
    train: Dataset({
        features: ['label', 'cont_X', 'cat_X'],
        num_rows: 79349
    })
    val: Dataset({
        features: ['label', 'cont_X', 'cat_X'],
        num_rows: 9919
    })
    test: Dataset({
        features: ['label', 'cont_X', 'cat_X'],
        num_rows: 9919
    })
})

In [18]:
datasets['train'][:2]

{'label': [[21998.0], [12995.0]],
 'cont_X': [[2019.0, 7237.0, 145.0, 60.099998474121094, 2.0],
  [2017.0, 36681.0, 30.0, 61.70000076293945, 1.7000000476837158]],
 'cat_X': [[2, 3, 0], [161, 1, 0]]}

In [19]:
datasets = datasets.with_format('torch')
datasets['train'][:2]

{'label': tensor([[21998.],
         [12995.]]),
 'cont_X': tensor([[2.0190e+03, 7.2370e+03, 1.4500e+02, 6.0100e+01, 2.0000e+00],
         [2.0170e+03, 3.6681e+04, 3.0000e+01, 6.1700e+01, 1.7000e+00]]),
 'cat_X': tensor([[  2,   3,   0],
         [161,   1,   0]])}

![](https://raw.githubusercontent.com/dvgodoy/assets/main/PyTorchInPractice/images/ch0/data_step5.png)

In [20]:
from torch.utils.data import DataLoader

dataloaders = {}
dataloaders['train'] = DataLoader(dataset=datasets['train'], batch_size=128, drop_last=True, shuffle=True)
dataloaders['val'] = DataLoader(dataset=datasets['val'], batch_size=128)
dataloaders['test'] = DataLoader(dataset=datasets['test'], batch_size=128)

In [21]:
next(iter(dataloaders['train']))

{'label': tensor([[ 8000.],
         [12200.],
         [10798.],
         [12700.],
         [13812.],
         [12490.],
         [14270.],
         [14400.],
         [ 2495.],
         [11200.],
         [ 9299.],
         [21841.],
         [ 7698.],
         [ 8250.],
         [11950.],
         [15199.],
         [ 7298.],
         [12990.],
         [15995.],
         [ 9750.],
         [18152.],
         [ 8498.],
         [22150.],
         [18990.],
         [27000.],
         [21990.],
         [48390.],
         [11495.],
         [ 9990.],
         [ 8790.],
         [48999.],
         [ 6290.],
         [ 5490.],
         [ 7240.],
         [19200.],
         [12990.],
         [19995.],
         [12495.],
         [13295.],
         [29532.],
         [10470.],
         [  675.],
         [32305.],
         [13989.],
         [10990.],
         [11998.],
         [49980.],
         [28900.],
         [23775.],
         [30490.],
         [20491.],
         [10998.],
   

### 5.3.2 BatchNorm for Continuous Attributes

![](https://raw.githubusercontent.com/dvgodoy/assets/main/PyTorchInPractice/images/ch0/model_step1.png)

In [22]:
import torch.nn as nn

batch = next(iter(dataloaders['train']))
batch['cont_X'].mean(axis=0), batch['cont_X'].std(axis=0, unbiased=False)

(tensor([2.0172e+03, 1.9635e+04, 1.2227e+02, 5.4502e+01, 1.7469e+00]),
 tensor([2.4805e+00, 2.0303e+04, 6.4556e+01, 2.2830e+01, 5.8509e-01]))

In [23]:
bn_layer = nn.BatchNorm1d(num_features=len(cont_attr))

normalized_cont = bn_layer(batch['cont_X'])
normalized_cont.mean(axis=0), normalized_cont.std(axis=0, unbiased=False)

(tensor([ 1.7544e-05,  1.8626e-08, -4.4703e-08, -5.6811e-08,  7.0781e-08],
        grad_fn=<MeanBackward1>),
 tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000], grad_fn=<StdBackward0>))

In [24]:
bn_layer.state_dict()

OrderedDict([('weight', tensor([1., 1., 1., 1., 1.])),
             ('bias', tensor([0., 0., 0., 0., 0.])),
             ('running_mean',
              tensor([2.0172e+02, 1.9635e+03, 1.2227e+01, 5.4502e+00, 1.7469e-01])),
             ('running_var',
              tensor([1.5201e+00, 4.1547e+07, 4.2093e+02, 5.3432e+01, 9.3450e-01])),
             ('num_batches_tracked', tensor(1))])

## 5.4 Lab 2: Price Prediction

## 5.5 Tour of High-Level Libraries

So far, we've been implementing everything ourselves, including a lot of boilerplate code such as the training loop and the early stopping. 

However, there are several high-level libraries built on top of PyTorch whose goal is, in general, to remove boilerplate and/or allow users to more easily leverage advanced capabilities such as mixed precision, distributed training, and more. 

Let's take a quick look at the most popular available libraries: HuggingFace Accelerate, Ignite, Catalyst, PyTorch Lightning, fast.ai, and Skorch.

### 5.5.1 HuggingFace Accelerate

[HuggingFace Accelerate](https://huggingface.co/docs/accelerate/index) is a library that allows you to leverage parallelization and distributed training with only a few lines of extra code added to your existing PyTorch workflow.

Here is a short example from its documentation (the plus signs indicated the lines added to the original code):

```python
+ from accelerate import Accelerator
+ accelerator = Accelerator()

+ model, optimizer, training_dataloader, scheduler = accelerator.prepare(
+     model, optimizer, training_dataloader, scheduler
+ )

  for batch in training_dataloader:
      optimizer.zero_grad()
      inputs, targets = batch
      inputs = inputs.to(device)
      targets = targets.to(device)
      outputs = model(inputs)
      loss = loss_function(outputs, targets)
+     accelerator.backward(loss)
      optimizer.step()
      scheduler.step()
```

For more details, check Accelerate's [migration](https://huggingface.co/docs/accelerate/basic_tutorials/migration) documentation.

### 5.5.2 Ignite

[PyTorch Ignite](https://pytorch-ignite.ai/) is a library focused on three high-level features: an engine and event system, out-of-the-box metrics for evaluation, and built-in handlers to composing pipelines, saving artifacts, and logging. Since it focuses on the training and validation pipelines, it means that your models, datasets, and optimizers remain in pure PyTorch.

Here's a short example from its documentation:

```python
# Setup training engine:
def train_step(engine, batch):
    # Users can do whatever they need on a single iteration
    # Eg. forward/backward pass for any number of models, optimizers, etc
    # ...

trainer = Engine(train_step)

# Setup single model evaluation engine
evaluator = create_supervised_evaluator(model, metrics={"accuracy": Accuracy()})

def validation():
    state = evaluator.run(validation_data_loader)
    # print computed metrics
    print(trainer.state.epoch, state.metrics)

# Run model's validation at the end of each epoch
trainer.add_event_handler(Events.EPOCH_COMPLETED, validation)

# Start the training
trainer.run(training_data_loader, max_epochs=100)
```

For more details, check Ignite's [migration](https://pytorch-ignite.ai/how-to-guides/02-convert-pytorch-to-ignite/) documentation and [code generator](https://code-generator.pytorch-ignite.ai/).

### 5.5.3 Catalyst

[Catalyst](https://catalyst-team.com/) focuses on reproducibility and rapid experimentation. It removes boilerplate code, improves readability, and offers scalability to any hardware without code changes. It is a deep learning framework and its basic building block is the Runner class, which takes care of the training loop.

Here's a short example from its documentation:
```python
runner = dl.SupervisedRunner(
    input_key="features", output_key="logits", target_key="targets", loss_key="loss"
)

# model training
runner.train(
    model=model,
    criterion=criterion,
    optimizer=optimizer,
    loaders=loaders,
    num_epochs=1,
    callbacks=[
        dl.AccuracyCallback(input_key="logits", target_key="targets", topk=(1, 3, 5)),
        dl.PrecisionRecallF1SupportCallback(input_key="logits", target_key="targets"),
    ],
    logdir="./logs",
    valid_loader="valid",
    valid_metric="loss",
    minimize_valid_metric=True,
    verbose=True,
)
```

For more details, check Catalyst's [quick start](https://catalyst-team.github.io/catalyst/getting_started/quickstart.html) documentation.

### 5.5.4 PyTorch Lightning

[PyTorch Lightning](https://www.pytorchlightning.ai/index.html) takes care of the engineering aspects of building and training a model in PyTorch. It is a framework itself, and its basic building block is the Lightning Module class, which acts as a model "recipe" that specifies all training details, and inherits from the typical PyTorch Module class. This means that, if you already have an implemented PyTorch workflow, your code will need to be refactored.

Here is a short example from its documentation:

```python
class LitAutoEncoder(pl.LightningModule):
	def __init__(self):
		super().__init__()
		self.encoder = nn.Sequential(
              nn.Linear(28 * 28, 64),
              nn.ReLU(),
              nn.Linear(64, 3))
		self.decoder = nn.Sequential(
              nn.Linear(3, 64),
              nn.ReLU(),
              nn.Linear(64, 28 * 28))

	def forward(self, x):
		embedding = self.encoder(x)
		return embedding

	def configure_optimizers(self):
		optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
		return optimizer

	def training_step(self, train_batch, batch_idx):
		x, y = train_batch
		x = x.view(x.size(0), -1)
		z = self.encoder(x)    
		x_hat = self.decoder(z)
		loss = F.mse_loss(x_hat, x)
		self.log('train_loss', loss)
		return loss

	def validation_step(self, val_batch, batch_idx):
		x, y = val_batch
		x = x.view(x.size(0), -1)
		z = self.encoder(x)
		x_hat = self.decoder(z)
		loss = F.mse_loss(x_hat, x)
		self.log('val_loss', loss)
```

For more details, check [this](https://github.com/Lightning-AI/lightning#pytorch-lightning-train-and-deploy-pytorch-at-scale) example of refactoring native PyTorch code into PyTorch Lightning.

### 5.5.5 fast.ai

[Fast.ai](https://docs.fast.ai/) is library that provides both high- and low- level components for practitioners to be rapidly productive and for researchers to hack it and configure it. Its high-level components include data loaders and learners, and fast.ai applications follow the same basic steps: creating data loaders, creating a learner, calling its `fit()` method, and making predictions.

Here is a short example from its documentation:

```python
path = untar_data(URLs.PETS)/'images'

def is_cat(x): return x[0].isupper()
dls = ImageDataLoaders.from_name_func(
    path, get_image_files(path), valid_pct=0.2, seed=42,
    label_func=is_cat, item_tfms=Resize(224))

learn = vision_learner(dls, resnet34, metrics=error_rate)
learn.fine_tune(1)

img = PILImage.create('images/cat.jpg')
is_cat,_,probs = learn.predict(img)
print(f"Is this a cat?: {is_cat}.")
print(f"Probability it's a cat: {probs[1].item():.6f}")
```

For more details, check fast.ai's [migration](https://docs.fast.ai/#migrating-from-other-libraries) documentation.

### 5.5.6 Skorch

[Skorch](https://github.com/skorch-dev/skorch) is a Scikit-Learn-compatible wrapper for PyTorch models. Its goal is to make it possible to use PyTorch with Sciki-Learn. It offers classes such as `NeuralNetClassifier` and `NeuralNetRegressor` to wrap your models that can then be used and trained like any other Scikit-Learn model.

Here'a a short example from its documentation:
```python
net = NeuralNetClassifier(
    MyModule,
    max_epochs=10,
    lr=0.1,
    # Shuffle training data on each epoch
    iterator_train__shuffle=True,
)

net.fit(X, y)
y_proba = net.predict_proba(X)
```

For more details, check Skorch's [documentation](https://skorch.readthedocs.io/en/latest/).